In [1]:
import requests
import tkinter as tk
import pandas as pd

In [72]:
import requests

class Money:
    '''
    Readme:

    This script defines the Money class, which facilitates currency conversion and basic arithmetic operations. It accesses exchange rates from an external API and offers methods to update and retrieve these rates. The Money class enables representation of monetary values in different currencies and supports operations on them.
    
    Usage:
    
    To use the Money class, follow these steps:
    
    1. Import the Money class into your Python script.
    2. Create an instance of the Money class by providing the initial amount and currency code.
    3. Use the various methods provided by the Money class to perform currency conversion and arithmetic operations.
    
    Dependencies:
    
    This script requires the requests library to make HTTP requests and retrieve data from the external API. No longer uses the pandas library for data manipulation.
    '''
    # Class-level variable to store exchange rates
    exchange_rates = None
    
    def __init__(self, amount, currency, sym_flag=False):
        """
        Initializes a Money object with the specified amount and currency.
        
        Args:
            amount (float): The amount of money.
            currency (str): The currency code (3-letter string of capitals).
            sym_flag (bool, optional): Whether to include currency symbols in the string representation.
            
        Raises:
            ValueError: If the currency is invalid or not uppercase.
        """
        # Fetch exchange rates if not already fetched
        if self.exchange_rates is None:
            self.fetch_x_rates()
            
        # Check if currency is valid
        if currency not in self.exchange_rates:
            raise ValueError("Invalid currency")

        # Check if currency is uppercase and three letters
        if not currency.isalpha() or len(currency) != 3 or not currency.isupper():
            raise ValueError("Currency must be a 3-letter string of capitals")

        # Convert amount to float and round to four decimal places
        self.amount = round(float(amount), 4)

        # Store currency
        self.currency = currency
        self.sym_flag = sym_flag
    
    @classmethod
    def fetch_x_rates(cls):
        """
        Fetches exchange rates from the API if they have not been fetched already.
        
        Returns:
            dict: Exchange rates for various currencies.
        """
        if cls.exchange_rates is None:
            url = 'https://api.exchangerate-api.com/v4/latest/EUR'
            response = requests.get(url)
            response.raise_for_status()
            cls.exchange_rates = response.json()['rates']
        return cls.exchange_rates

    @classmethod
    def get_x_rates(cls):
        """
        Returns the exchange rates if already fetched, otherwise fetches and returns them.
        
        Returns:
            dict: Exchange rates for various currencies.
        """
        if cls.exchange_rates is None:
            cls.fetch_x_rates()
        return cls.exchange_rates
        
    @classmethod
    def update_x_rates(cls):
        """
        Updates the exchange rates by fetching the latest rates from the API.
        """
        url = 'https://api.exchangerate-api.com/v4/latest/USD'
        response = requests.get(url)
        response.raise_for_status()
        cls.exchange_rates = response.json()['rates']

    def __str__(self):
        """
        Returns a string representation of the Money object.
        
        Returns:
            str: String representation of the Money object.
        """
        currency_symbols = {'USD': '$',   # US Dollar
        'EUR': '€',   # Euro
        'GBP': '£',   # British Pound Sterling
        'JPY': '¥',   # Japanese Yen
        'INR': '₹',   # Indian Rupee
        'AUD': 'A$',  # Australian Dollar
        'CAD': 'C$',  # Canadian Dollar
        'CHF': 'CHF', # Swiss Franc
        'CNY': '元',   # Chinese Yuan
        'SEK': 'kr',  # Swedish Krona
        'NZD': 'NZ$', # New Zealand Dollar
        'KRW': '₩',   # South Korean Won
        'SGD': 'S$',  # Singapore Dollar
        'NOK': 'kr',  # Norwegian Krone
        'MXN': 'Mex$', # Mexican Peso
        'BRL': 'R$',   # Brazilian Real
        'RUB': '₽',   # Russian Ruble
        'ZAR': 'R',    # South African Rand
        'TRY': '₺',    # Turkish Lira
        'HKD': 'HK$'}  # Hong Kong Dollar}
        
        if self.sym_flag:
            return f"{currency_symbols[self.currency]} {self.amount:.4f}"
        else:
            return f"{self.currency} {self.amount:.4f}"
            

    def convert_to(self, other_currency, exchange_rates=None):
        """
        Converts the amount to the specified currency.
        
        Args:
            other_currency (str): The currency code to convert to.
            exchange_rates (dict, optional): Optional exchange rates to use for conversion.
            
        Returns:
            Money: Converted amount in currency.
            
        Raises:
            ValueError: If the other_currency is not in the exchange rates.
        """
        # Retrieve exchange rates
        if exchange_rates:
            self.exchange_rates = exchange_rates
        else:
            self.exchange_rates = self.fetch_x_rates()
        
        # Perform currency conversion
        if other_currency == self.currency:
            return self
        return Money(self.amount / self.exchange_rates[self.currency] * self.exchange_rates[other_currency],
                     other_currency, self.sym_flag)

    def __add__(self, other):
        """
        Adds two Money objects together.
        
        Args:
            other (Money): The other Money object to add.
            
        Returns:
            Money: Result of the addition operation.
            
        Raises:
            ValueError: If other is not a Money object.
        """
        if isinstance(other, Money):
            other_same_currency = other.convert_to(self.currency)
            other_amount = other_same_currency.amount
            return Money(self.amount + other_amount, self.currency, self.sym_flag)
        raise ValueError("Can only add Money objects")

    def __sub__(self, other):
        """
        Subtracts one Money object from another.
        
        Args:
            other (Money): The other Money object to subtract.
            
        Returns:
            Money: Result of the subtraction operation.
            
        Raises:
            ValueError: If other is not a Money object.
        """
        if isinstance(other, Money):
            other_same_currency = other.convert_to(self.currency)
            other_amount = other_same_currency.amount
            return Money(self.amount - other_amount, self.currency, self.sym_flag)
        raise ValueError("Can only subtract Money objects")

    def __mul__(self, constant):
        """
        Multiplies the amount by a constant.
        
        Args:
            constant (int, float): The constant to multiply by.
            
        Returns:
            Money: Result of the multiplication operation.
            
        Raises:
            ValueError: If constant is not an int or float.
        """
        if isinstance(constant, (int, float)):
            return Money(self.amount * constant, self.currency, self.sym_flag)
       
    def __truediv__(self, divisor):
        """
        Divides the amount by a divisor.
        
        Args:
            divisor (int, float): The value to divide the amount by.
            
        Returns:
            Money: Result of the division operation.
            
        Raises:
            ValueError: If divisor is zero.
        """
        if divisor == 0:
            raise ValueError("Division by zero")
        return Money(self.amount / divisor, self.currency,self.sym_flag)

$ 100.0000
€ 32.0000
134.56 USD
65.44 USD
200.0 USD
50.0 USD
92.5926 EUR
$ 34.5600
$ 34.5600
